# Statistical Arbitrage Strategy: Reversal and Momentum
## A Capstone Project in Quantitative Finance

### Introduction
This project focuses on developing and implementing a statistical arbitrage trading strategy that combines reversal and momentum signals. The goal is to create a quant strategy that exploits price inefficiencies in financial markets, simulating a real-world quant work experience. The strategy will be backtested, optimized, and tested in a live trading environment.

### Objectives
- Develop a statistical arbitrage strategy using reversal and momentum indicators.
- Backtest the strategy on historical data to evaluate its performance.
- Optimize the strategy parameters to improve profitability and manage risk.
- Implement the strategy in a live trading environment to validate its effectiveness.

### Project Structure
1. **Strategy Blueprint**: Define the core strategy, including objectives, hypotheses, and methodology.
2. **Data Sourcing**: Gather and prepare the necessary historical price data.
3. **Signal Construction**: Create reversal and momentum signals based on selected indicators.
4. **Backtesting Framework**: Simulate the strategy using historical data and evaluate its performance.
5. **Performance Evaluation**: Analyze key metrics such as Sharpe Ratio, Maximum Drawdown, and cumulative returns.
6. **Weighting and Portfolio Construction**: Develop a portfolio based on optimized signal weights.
7. **Optimization**: Fine-tune strategy parameters to maximize performance.
8. **Trading Costs**: Incorporate realistic trading costs and assess their impact on profitability.
9. **Live Trading**: Set up and run the strategy in a live trading environment.
10. **Final Presentation**: Compile and present the project findings.

### Environment Setup
Before diving into the analysis, we will import the necessary libraries and set up the environment for data analysis and visualization.


In [1]:
# Importing essential libraries
import numpy as np
import scipy.stats as stats
import pandas as pd
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import simpy
import cvxpy as cp
from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA
import streamlit as st
import yfinance as yf
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
import yaml
from fredapi import Fred
import warnings
warnings.filterwarnings('ignore')

# Set plotting styles
# plt.style.use('seaborn-darkgrid')

# Display options for better readability
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 50)

# Notebook title
print("Statistical Arbitrage Strategy: Reversal and Momentum - Capstone Project")

Statistical Arbitrage Strategy: Reversal and Momentum - Capstone Project


## Load FRED API

In [2]:
# Load the .yaml file
with open('C:\\Users\\blake\\Documents\\github\\finance\\quant\\fred\\fred_api_key.yml', 'r') as file:
    config = yaml.safe_load(file)

# Access the FRED API key from the .yaml file
fred_api_key = config['FRED_API_KEY']

# Initialize the FRED API with the API key
fred = Fred(api_key=fred_api_key)

# Pull data for S&P 500, interest rates, CPI, unemployment, GDP, and additional indicators
sp500_data = fred.get_series('SP500')
interest_rate_data = fred.get_series('FEDFUNDS')
cpi_data = fred.get_series('CPIAUCSL')
unemployment_data = fred.get_series('UNRATE')
gdp_data = fred.get_series('GDP')
industrial_production_data = fred.get_series('INDPRO')
treasury_yield_10yr_data = fred.get_series('GS10')
vix_data = fred.get_series('VIXCLS')
pce_data = fred.get_series('PCEPI')
nonfarm_payrolls_data = fred.get_series('PAYEMS')
consumer_confidence_data = fred.get_series('UMCSENT')
housing_starts_data = fred.get_series('HOUST')

# Combine data into a single DataFrame
df_mac = pd.DataFrame({
    'S&P500': sp500_data,
    'Interest_Rate': interest_rate_data,
    'CPI': cpi_data,
    'Unemployment_Rate': unemployment_data,
    'GDP': gdp_data,
    'Industrial_Production': industrial_production_data,
    '10Y_Treasury_Yield': treasury_yield_10yr_data,
    'VIX': vix_data,
    'PCE': pce_data,
    'Nonfarm_Payrolls': nonfarm_payrolls_data,
    'Consumer_Confidence': consumer_confidence_data,
    'Housing_Starts': housing_starts_data
}).dropna()

df_mac.head()

,S&P500,Interest_Rate,CPI,Unemployment_Rate,GDP,Industrial_Production,10Y_Treasury_Yield,VIX,PCE,Nonfarm_Payrolls,Consumer_Confidence,Housing_Starts
2014-10-01,1946.16,0.09,237.430,5.7,17912.079,102.9611,2.30,16.71,97.384,139804.0,86.9,1074.0
2015-04-01,2059.69,0.12,236.222,5.4,18279.784,101.2255,1.94,15.11,97.094,141202.0,95.9,1190.0
2015-07-01,2077.42,0.13,238.034,5.2,18401.626,101.0893,2.32,16.09,97.605,141989.0,93.1,1146.0
2015-10-01,1923.82,0.12,237.733,5.0,18435.137,100.1563,2.07,22.55,97.477,142584.0,90.0,1058.0
2016-04-01,2072.78,0.37,238.992,5.1,18711.702,98.4775,1.81,13.10,97.993,143856.0,89.0,1163.0


## S&P 500 Data

In [3]:
download_dir = "D:\\datasets\\github_financial_time_series_analysis"
api = KaggleApi()
api.authenticate()

dataset = 'andrewmvd/sp-500-stocks'
api.dataset_download_files(dataset, path=download_dir, unzip=True)

files_of_interest = ['sp500_stocks.csv', 'sp500_companies.csv', 'sp500_index.csv']
for file_name in files_of_interest:
    file_path = os.path.join(download_dir, file_name)
    if os.path.exists(file_path):
        print(f'{file_name} has been downloaded successfully.')
    else:
        print(f'{file_name} is missing.')

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks
sp500_stocks.csv has been downloaded successfully.
sp500_companies.csv has been downloaded successfully.
sp500_index.csv has been downloaded successfully.


In [4]:
df = pd.read_csv("D:\\datasets\\github_financial_time_series_analysis\\sp500_stocks.csv")
df_sym = pd.read_csv("D:\\datasets\\github_financial_time_series_analysis\\sp500_companies.csv")
df_ind = pd.read_csv("D:\\datasets\\github_financial_time_series_analysis\\sp500_index.csv")

## Define Stock Tickers

In [5]:
# Blue-Chip Stocks
blue_chip_tickers = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'JNJ', 'PG', 'V']

# Sector-Specific Leaders
sector_leaders_tickers = ['TSLA', 'NVDA', 'JPM', 'BA', 'XOM']

# Emerging Growth Stocks
emerging_growth_tickers = ['SHOP', 'SQ', 'ROKU']

# ETFs
etf_tickers = ['SPY', 'QQQ', 'EEM']

# Dividends and Defensive Stocks
dividends_tickers = ['KO', 'PEP', 'T']

# Sports & Media Stocks
sports_media_tickers = ['MANU']  # Manchester United

# Combine all tickers into one list
all_tickers = blue_chip_tickers + sector_leaders_tickers + emerging_growth_tickers + etf_tickers + dividends_tickers + sports_media_tickers

## Yahoo Finance Data

In [6]:
# Download historical data for all tickers
df_yf = yf.download(all_tickers, start="2015-01-01", end="2024-12-31", group_by='ticker') 

[**********************77%************           ]  17 of 22 completed2024-09-05 18:59:23,821 WARNING Connection pool is full, discarding connection: query2.finance.yahoo.com. Connection pool size: 10
[**********************82%**************         ]  18 of 22 completed2024-09-05 18:59:23,877 WARNING Connection pool is full, discarding connection: query2.finance.yahoo.com. Connection pool size: 10
[*********************100%***********************]  22 of 22 completed


In [7]:
df_yf.tail()

Ticker                          ROKU                                   \
Price                           Open       High        Low      Close   
Date                                                                    
2024-08-29 00:00:00+00:00  67.449997  69.750000  67.360001  68.440002   
2024-08-30 00:00:00+00:00  68.699997  68.830002  63.830002  67.769997   
2024-09-03 00:00:00+00:00  67.639999  67.639999  64.910004  65.220001   
2024-09-04 00:00:00+00:00  64.370003  65.930000  63.279999  63.549999   
2024-09-05 00:00:00+00:00  66.849998  68.390999  65.386597  66.699997   

Ticker                                                GOOGL              \
Price                      Adj Close     Volume        Open        High   
Date                                                                      
2024-08-29 00:00:00+00:00  68.440002  3398300.0  164.309998  165.970001   
2024-08-30 00:00:00+00:00  67.769997  6091000.0  162.619995  163.660004   
2024-09-03 00:00:00+00:00  65.220001  2339300.0  161.720001  161.850006   
2024-09-04 00:00:00+00:00  63.549999  2276200.0  156.660004  159.000000   
2024-09-05 00:00:00+00:00  66.699997  4943108.0  156.149994  159.449997   

Ticker                                             ...         PEP  \
Price                             Low       Close  ...         Low   
Date                                               ...               
2024-08-29 00:00:00+00:00  160.250000  161.779999  ...  171.910004   
2024-08-30 00:00:00+00:00  161.690002  163.380005  ...  171.250000   
2024-09-03 00:00:00+00:00  156.479996  157.360001  ...  171.880005   
2024-09-04 00:00:00+00:00  155.960007  156.449997  ...  176.289993   
2024-09-05 00:00:00+00:00  155.980499  157.240005  ...  177.520004   

Ticker                                                             KO  \
Price                           Close   Adj Close   Volume       Open   
Date                                                                    
2024-08-29 00:00:00+00:00  172.089996  172.089996  4654300  71.900002   
2024-08-30 00:00:00+00:00  172.880005  172.880005  8677300  72.199997   
2024-09-03 00:00:00+00:00  177.539993  177.539993  7031400  72.440002   
2024-09-04 00:00:00+00:00  178.520004  178.520004  6188500  73.300003   
2024-09-05 00:00:00+00:00  179.300003  179.300003  5734220  72.620003   

Ticker                                                                 \
Price                           High        Low      Close  Adj Close   
Date                                                                    
2024-08-29 00:00:00+00:00  72.220001  71.230003  72.050003  72.050003   
2024-08-30 00:00:00+00:00  72.570000  71.779999  72.470001  72.470001   
2024-09-03 00:00:00+00:00  73.290001  72.250000  73.010002  73.010002   
2024-09-04 00:00:00+00:00  73.529999  71.910004  72.559998  72.559998   
2024-09-05 00:00:00+00:00  72.669998  70.980003  71.169998  71.169998   

Ticker                               
Price                        Volume  
Date                                 
2024-08-29 00:00:00+00:00  17201700  
2024-08-30 00:00:00+00:00  18624200  
2024-09-03 00:00:00+00:00  14751900  
2024-09-04 00:00:00+00:00  18140800  
2024-09-05 00:00:00+00:00  17719744  

[5 rows x 132 columns]

In [8]:
df_yf.columns

MultiIndex([( 'ROKU',      'Open'),
            ( 'ROKU',      'High'),
            ( 'ROKU',       'Low'),
            ( 'ROKU',     'Close'),
            ( 'ROKU', 'Adj Close'),
            ( 'ROKU',    'Volume'),
            ('GOOGL',      'Open'),
            ('GOOGL',      'High'),
            ('GOOGL',       'Low'),
            ('GOOGL',     'Close'),
            ...
            (  'PEP',       'Low'),
            (  'PEP',     'Close'),
            (  'PEP', 'Adj Close'),
            (  'PEP',    'Volume'),
            (   'KO',      'Open'),
            (   'KO',      'High'),
            (   'KO',       'Low'),
            (   'KO',     'Close'),
            (   'KO', 'Adj Close'),
            (   'KO',    'Volume')],
           names=['Ticker', 'Price'], length=132)

In [9]:
# Convert to long format
df_long = df_yf.stack(level=0).reset_index().rename(columns={'level_1': 'Ticker'})

In [10]:
df_long.tail()

Price,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
52557,2024-09-05 00:00:00+00:00,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0
52558,2024-09-05 00:00:00+00:00,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0
52559,2024-09-05 00:00:00+00:00,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0
52560,2024-09-05 00:00:00+00:00,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0
52561,2024-09-05 00:00:00+00:00,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0


In [11]:
df_long.columns

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close',
       'Volume'],
      dtype='object', name='Price')

In [12]:
df_long.reset_index(drop=True, inplace=True)

In [13]:
df_long.tail()

Price,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
52557,2024-09-05 00:00:00+00:00,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0
52558,2024-09-05 00:00:00+00:00,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0
52559,2024-09-05 00:00:00+00:00,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0
52560,2024-09-05 00:00:00+00:00,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0
52561,2024-09-05 00:00:00+00:00,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0


In [14]:
df_long.index.name = None

In [15]:
df_long.set_index('Date', inplace=True)

In [16]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2024-09-05 00:00:00+00:00,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0
2024-09-05 00:00:00+00:00,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0
2024-09-05 00:00:00+00:00,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0
2024-09-05 00:00:00+00:00,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0
2024-09-05 00:00:00+00:00,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0


In [17]:
df_long.sort_index(inplace=True)

In [18]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2024-09-05 00:00:00+00:00,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0
2024-09-05 00:00:00+00:00,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0
2024-09-05 00:00:00+00:00,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0
2024-09-05 00:00:00+00:00,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0
2024-09-05 00:00:00+00:00,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0


In [19]:
print(df_long.index)
print("\n")
print(df_mac.index)

DatetimeIndex(['2015-01-02 00:00:00+00:00', '2015-01-02 00:00:00+00:00',
               '2015-01-02 00:00:00+00:00', '2015-01-02 00:00:00+00:00',
               '2015-01-02 00:00:00+00:00', '2015-01-02 00:00:00+00:00',
               '2015-01-02 00:00:00+00:00', '2015-01-02 00:00:00+00:00',
               '2015-01-02 00:00:00+00:00', '2015-01-02 00:00:00+00:00',
               ...
               '2024-09-05 00:00:00+00:00', '2024-09-05 00:00:00+00:00',
               '2024-09-05 00:00:00+00:00', '2024-09-05 00:00:00+00:00',
               '2024-09-05 00:00:00+00:00', '2024-09-05 00:00:00+00:00',
               '2024-09-05 00:00:00+00:00', '2024-09-05 00:00:00+00:00',
               '2024-09-05 00:00:00+00:00', '2024-09-05 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=52562, freq=None)


DatetimeIndex(['2014-10-01', '2015-04-01', '2015-07-01', '2015-10-01',
               '2016-04-01', '2016-07-01', '2018-10-01', '2019-04-01',
               '2019-07-0

In [20]:
# Convert both dataframes to timezone-naive index
df_long.index = df_long.index.tz_localize(None)
df_mac.index = df_mac.index.tz_localize(None)

In [21]:
# Adjust the stock data to have the same index as the economic data (if needed)
df_comb = df_mac.join(df_long, how='outer')

In [22]:
df_comb.head()

,S&P500,Interest_Rate,CPI,Unemployment_Rate,GDP,Industrial_Production,10Y_Treasury_Yield,VIX,PCE,Nonfarm_Payrolls,Consumer_Confidence,Housing_Starts,Ticker,Open,High,Low,Close,Adj Close,Volume
2014-10-01,1946.16,0.09,237.43,5.7,17912.079,102.9611,2.3,16.71,97.384,139804.0,86.9,1074.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAPL,27.847500,27.860001,26.837500,27.332500,24.373962,212818400.0
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMZN,15.629000,15.737500,15.348000,15.426000,15.426000,55664000.0
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BA,131.070007,131.839996,129.089996,129.949997,113.657219,4294200.0
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EEM,39.189999,39.209999,38.669998,38.770000,31.612551,65010000.0


In [23]:
columns_to_move = ['Ticker']

# Remaining columns
remaining_columns = [col for col in df_comb.columns if col not in columns_to_move]

# Insert columns at the second position (index 1)
insert_position = 0
new_order = remaining_columns[:insert_position] + columns_to_move + remaining_columns[insert_position:]

# Reorder the DataFrame
df_comb = df_comb[new_order]

In [24]:
df_comb

,Ticker,S&P500,Interest_Rate,CPI,Unemployment_Rate,GDP,Industrial_Production,10Y_Treasury_Yield,VIX,PCE,Nonfarm_Payrolls,Consumer_Confidence,Housing_Starts,Open,High,Low,Close,Adj Close,Volume
2014-10-01,NaN,1946.16,0.09,237.43,5.7,17912.079,102.9611,2.3,16.71,97.384,139804.0,86.9,1074.0,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-02,AAPL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.847500,27.860001,26.837500,27.332500,24.373962,212818400.0
2015-01-02,AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.629000,15.737500,15.348000,15.426000,15.426000,55664000.0
2015-01-02,BA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.070007,131.839996,129.089996,129.949997,113.657219,4294200.0
2015-01-02,EEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.189999,39.209999,38.669998,38.770000,31.612551,65010000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-05,SQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0
2024-09-05,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0
2024-09-05,TSLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0
2024-09-05,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0


In [25]:
%who

ARIMA	 Fred	 KaggleApi	 LinearRegression	 all_tickers	 api	 arch_model	 blue_chip_tickers	 columns_to_move	 
config	 consumer_confidence_data	 cp	 cpi_data	 dataset	 datetime	 df	 df_comb	 df_ind	 
df_long	 df_mac	 df_sym	 df_yf	 dividends_tickers	 download_dir	 emerging_growth_tickers	 etf_tickers	 file	 
file_name	 file_path	 files_of_interest	 fred	 fred_api_key	 gdp_data	 go	 housing_starts_data	 industrial_production_data	 
insert_position	 interest_rate_data	 new_order	 nonfarm_payrolls_data	 np	 os	 pce_data	 pd	 plt	 
px	 remaining_columns	 seasonal_decompose	 sector_leaders_tickers	 simpy	 sns	 sp500_data	 sports_media_tickers	 st	 
stats	 treasury_yield_10yr_data	 unemployment_data	 vix_data	 warnings	 yaml	 yf	 zipfile	 


In [26]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0
2024-09-05,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0
2024-09-05,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0


## Use 'df_long' df

In [27]:
df_long['returns'] = df_long['Adj Close'].pct_change()  # Calculate daily returns

In [28]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns
Date,,,,,,,,
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247
2024-09-05,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0,0.210497
2024-09-05,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0,-0.593820


## Part III: Signal Construction

In [34]:
df_long.rename(columns = {'Price': 'price', 'Date': 'date', 'Ticker': 'ticker', 'Close': 'close', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Adj Close': 'adj_close', 'Volume': 'volume'})

Price,ticker,open,high,low,close,adj_close,volume,returns
Date,,,,,,,,
2015-01-02,AAPL,27.847500,27.860001,26.837500,27.332500,24.373962,212818400.0,NaN
2015-01-02,AMZN,15.629000,15.737500,15.348000,15.426000,15.426000,55664000.0,-0.367112
2015-01-02,BA,131.070007,131.839996,129.089996,129.949997,113.657219,4294200.0,6.367900
2015-01-02,EEM,39.189999,39.209999,38.669998,38.770000,31.612551,65010000.0,-0.721861
2015-01-02,GOOGL,26.629999,26.790001,26.393999,26.477501,26.447147,26480000.0,-0.163397
...,...,...,...,...,...,...,...,...
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247


In [35]:
df_long.columns

Index(['Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'returns'],
      dtype='object', name='Price')

In [36]:
df_long['MA50'] = df_long['Close'].rolling(window=50).mean()
df_long['MA200'] = df_long['Close'].rolling(window=200).mean()
df_long['Signal'] = 0
df_long.loc[df_long['MA50'] > df_long['MA200'], 'Signal'] = 1  # Buy
df_long.loc[df_long['MA50'] < df_long['MA200'], 'Signal'] = -1  # Sell

In [37]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,Signal
Date,,,,,,,,,,,
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498,185.516201,182.332375,1
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242,182.438801,179.654525,1
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247,177.806001,180.486175,-1
2024-09-05,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0,0.210497,182.074001,181.781725,1
2024-09-05,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0,-0.593820,182.912201,181.294275,1


In [38]:
def compute_RSI(data, window=14):
    delta = data.diff(1)  # Calculate the difference between consecutive prices
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()  # Gains (positive price changes)
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()  # Losses (negative price changes)
    
    RS = gain / loss  # Relative Strength
    RSI = 100 - (100 / (1 + RS))  # Relative Strength Index

    return RSI

In [39]:
df_long['RSI'] = compute_RSI(df_long['Close'], window=14)
df_long['reversal_signal'] = 0
df_long.loc[df_long['RSI'] > 70, 'reversal_signal'] = -1  # Overbought - Sell
df_long.loc[df_long['RSI'] < 30, 'reversal_signal'] = 1   # Oversold - Buy

In [40]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,Signal,RSI,reversal_signal
Date,,,,,,,,,,,,,
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498,185.516201,182.332375,1,50.394358,0
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242,182.438801,179.654525,1,47.490197,0
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247,177.806001,180.486175,-1,51.114988,0
2024-09-05,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0,0.210497,182.074001,181.781725,1,51.044811,0
2024-09-05,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0,-0.593820,182.912201,181.294275,1,50.714845,0


In [41]:
def bollinger_bands(data, window=20, std_dev=2):
    # Calculate the moving average (middle band)
    moving_avg = data.rolling(window=window).mean()
    
    # Calculate the standard deviation
    rolling_std = data.rolling(window=window).std()
    
    # Calculate the upper and lower Bollinger Bands
    upper_band = moving_avg + (rolling_std * std_dev)
    lower_band = moving_avg - (rolling_std * std_dev)
    
    return upper_band, lower_band

In [42]:
df_long.head()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,Signal,RSI,reversal_signal
Date,,,,,,,,,,,,,
2015-01-02,AAPL,27.847500,27.860001,26.837500,27.332500,24.373962,212818400.0,NaN,NaN,NaN,0,NaN,0
2015-01-02,AMZN,15.629000,15.737500,15.348000,15.426000,15.426000,55664000.0,-0.367112,NaN,NaN,0,NaN,0
2015-01-02,BA,131.070007,131.839996,129.089996,129.949997,113.657219,4294200.0,6.367900,NaN,NaN,0,NaN,0
2015-01-02,EEM,39.189999,39.209999,38.669998,38.770000,31.612551,65010000.0,-0.721861,NaN,NaN,0,NaN,0
2015-01-02,GOOGL,26.629999,26.790001,26.393999,26.477501,26.447147,26480000.0,-0.163397,NaN,NaN,0,NaN,0


In [43]:
df_long['rolling_mean'] = df_long.groupby(df_long.index)['Adj Close'].rolling(window=20, min_periods=1).mean().reset_index(level=0, drop=True)

In [44]:
df_long.sort_index(inplace=True)  # Sort by the date index before applying rolling functions

In [45]:
df_long['time_based_rolling'] = df_long['Adj Close'].rolling(window='20D').mean()  # Rolling based on 20-day periods

In [46]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,Signal,RSI,reversal_signal,rolling_mean,time_based_rolling
Date,,,,,,,,,,,,,,,
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498,185.516201,182.332375,1,50.394358,0,184.194446,181.769009
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242,182.438801,179.654525,1,47.490197,0,175.586843,181.199684
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247,177.806001,180.486175,-1,51.114988,0,178.316001,181.372115
2024-09-05,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0,0.210497,182.074001,181.781725,1,51.044811,0,181.128001,181.713335
2024-09-05,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0,-0.593820,182.912201,181.294275,1,50.714845,0,177.892001,181.473673


In [47]:
df_long['upper_BB'], df_long['lower_BB'] = bollinger_bands(df_long['Adj Close'], window=20, std_dev=2)
# df_long.loc[df_long['Close'] > df_long['upper_BB'], 'reversal_signal'] = -1  # Sell
# df_long.loc[df_long['Close'] < df_long['lower_BB'], 'reversal_signal'] = 1   # Buy

In [48]:
df_long.tail(19)

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,Signal,RSI,reversal_signal,rolling_mean,time_based_rolling,upper_BB,lower_BB
Date,,,,,,,,,,,,,,,,,
2024-09-05,EEM,42.520000,42.724998,42.430000,42.560001,42.560001,19989731.0,-0.737527,177.840401,180.488575,-1,47.187027,0,151.245000,181.150525,477.046954,-114.298956
2024-09-05,GOOGL,156.149994,159.449997,155.980499,157.240005,157.240005,18599653.0,2.694549,175.457801,180.410975,-1,49.631039,0,152.444001,181.061638,473.568060,-117.029062
2024-09-05,JNJ,167.199997,167.300003,164.830002,164.990005,164.990005,3475848.0,0.049288,176.398801,181.020325,-1,43.435802,0,154.535002,181.002113,474.088372,-108.306373
2024-09-05,JPM,220.149994,220.755005,216.033707,217.630005,217.630005,8008430.0,0.319050,176.296001,181.289475,-1,54.039133,0,163.548574,181.137272,473.705425,-87.874425
2024-09-05,KO,72.620003,72.669998,70.980003,71.169998,71.169998,17719744.0,-0.672977,174.194401,180.833575,-1,50.023203,0,152.001252,180.732980,442.378099,-90.320099
2024-09-05,MANU,17.219999,17.325001,17.000000,17.080000,17.080000,200940.0,-0.760011,171.315601,179.835825,-1,33.527513,0,137.010002,180.133519,445.709092,-102.564091
2024-09-05,MSFT,408.204987,413.100006,406.130005,408.390015,408.390015,14089837.0,22.910423,178.633201,181.531125,-1,61.289371,0,164.148003,180.966572,476.990048,-110.858047
2024-09-05,NVDA,104.980003,109.650002,104.760002,107.209999,107.209999,298016028.0,-0.737481,177.630201,181.979025,-1,52.430158,0,158.971821,180.698366,475.506922,-116.243920
2024-09-05,PEP,178.729996,179.729996,177.520004,179.300003,179.300003,5734220.0,0.672419,177.873001,180.797775,-1,48.788840,0,160.665836,180.693299,430.308257,-99.176254


In [49]:
def compute_MACD(data, fast=12, slow=26, signal=9):
    # Calculate the fast (short-term) and slow (long-term) EMAs
    ema_fast = data.ewm(span=fast, adjust=False).mean()
    ema_slow = data.ewm(span=slow, adjust=False).mean()
    
    # Calculate the MACD line
    MACD = ema_fast - ema_slow
    
    # Calculate the Signal line (9-day EMA of the MACD)
    signal_line = MACD.ewm(span=signal, adjust=False).mean()
    
    return MACD, signal_line

In [50]:
df_long['MACD'], df_long['Signal_Line'] = compute_MACD(df_long['Close'], fast=12, slow=26, signal=9)
df_long['Momentum_Signal'] = 0
df_long.loc[df_long['MACD'] > df_long['Signal_Line'], 'Momentum_Signal'] = 1   # Buy
df_long.loc[df_long['MACD'] < df_long['Signal_Line'], 'Momentum_Signal'] = -1  # Sell

In [51]:
df_long.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,Signal,RSI,reversal_signal,rolling_mean,time_based_rolling,upper_BB,lower_BB,MACD,Signal_Line,Momentum_Signal
Date,,,,,,,,,,,,,,,,,,,,
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498,185.516201,182.332375,1,50.394358,0,184.194446,181.769009,470.830307,-99.825305,12.561939,7.563563,1
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242,182.438801,179.654525,1,47.490197,0,175.586843,181.199684,463.239052,-118.218049,-3.083380,5.434175,-1
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247,177.806001,180.486175,-1,51.114988,0,178.316001,181.372115,468.767105,-112.135103,1.407875,4.628915,-1
2024-09-05,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0,0.210497,182.074001,181.781725,1,51.044811,0,181.128001,181.713335,474.450121,-112.194120,8.775577,5.458247,1
2024-09-05,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0,-0.593820,182.912201,181.294275,1,50.714845,0,177.892001,181.473673,472.788320,-117.004319,1.249697,4.616537,-1


In [52]:
df_long.describe()

Price,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,Signal,RSI,reversal_signal,rolling_mean,time_based_rolling,upper_BB,lower_BB,MACD,Signal_Line,Momentum_Signal
count,52562.000000,52562.000000,52562.000000,52562.000000,52562.000000,5.256200e+04,52561.000000,52513.000000,52363.000000,52562.000000,52549.000000,52562.0,52562.000000,52562.000000,52543.000000,52543.000000,52562.000000,52562.000000,52562.000000
mean,113.532203,114.727183,112.296897,113.540627,107.471203,5.518797e+07,3.063833,113.533238,113.509475,0.010140,49.955165,0.0,104.048473,107.001265,276.286479,-61.352287,0.020847,0.020496,-0.177448
std,98.454562,99.348749,97.498452,98.448867,97.128423,1.163280e+08,12.457457,40.404743,40.166630,0.998063,4.597839,0.0,41.488137,42.599679,100.024984,20.546300,8.514466,4.102549,0.984130
min,0.481250,0.487500,0.473500,0.478500,0.459453,0.000000e+00,-0.988449,52.256228,54.883043,-1.000000,32.709920,0.0,19.080944,19.899981,100.558660,-134.163495,-28.251766,-15.946469,-1.000000
25%,40.189999,40.540001,39.820000,40.211249,36.141098,7.399900e+06,-0.627422,76.479137,77.521926,-1.000000,47.735195,0.0,65.913726,66.824004,181.289285,-76.168017,-4.742449,-2.008951,-1.000000
50%,87.924999,88.830002,87.000000,87.936253,77.224796,1.915475e+07,-0.093220,106.411073,105.835595,1.000000,49.842905,0.0,105.079745,99.057831,272.731068,-61.125762,-1.106925,0.100693,-1.000000
75%,156.869995,158.561119,155.399994,156.913380,151.101719,5.359265e+07,1.355357,151.790780,152.792777,1.000000,52.077082,0.0,136.661648,148.381511,361.835332,-41.970021,4.258804,2.254662,1.000000
max,563.179993,565.159973,562.099976,564.859985,564.859985,3.692928e+09,158.306169,197.211346,187.539450,1.000000,64.530132,0.0,234.548523,185.931746,506.538349,-20.441265,38.063281,15.245711,1.000000


In [53]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52562 entries, 2015-01-02 to 2024-09-05
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ticker              52562 non-null  object 
 1   Open                52562 non-null  float64
 2   High                52562 non-null  float64
 3   Low                 52562 non-null  float64
 4   Close               52562 non-null  float64
 5   Adj Close           52562 non-null  float64
 6   Volume              52562 non-null  float64
 7   returns             52561 non-null  float64
 8   MA50                52513 non-null  float64
 9   MA200               52363 non-null  float64
 10  Signal              52562 non-null  int64  
 11  RSI                 52549 non-null  float64
 12  reversal_signal     52562 non-null  int64  
 13  rolling_mean        52562 non-null  float64
 14  time_based_rolling  52562 non-null  float64
 15  upper_BB            52543 non-null  

In [54]:
# Initialize a new column for signals
df_long['MACD_Signal'] = 0  # Default to no position

# Generate Buy Signal (MACD crosses above Signal Line)
df_long.loc[df_long['MACD'] > df_long['Signal_Line'], 'MACD_Signal'] = 1  # Buy

# Generate Sell Signal (MACD crosses below Signal Line)
df_long.loc[df_long['MACD'] < df_long['Signal_Line'], 'MACD_Signal'] = -1  # Sell

In [55]:
df_long.head()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,...,RSI,reversal_signal,rolling_mean,time_based_rolling,upper_BB,lower_BB,MACD,Signal_Line,Momentum_Signal,MACD_Signal
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,AAPL,27.847500,27.860001,26.837500,27.332500,24.373962,212818400.0,NaN,NaN,NaN,...,NaN,0,24.373962,24.373962,NaN,NaN,0.000000,0.000000,0,0
2015-01-02,AMZN,15.629000,15.737500,15.348000,15.426000,15.426000,55664000.0,-0.367112,NaN,NaN,...,NaN,0,19.899981,19.899981,NaN,NaN,-0.949806,-0.189961,-1,-1
2015-01-02,BA,131.070007,131.839996,129.089996,129.949997,113.657219,4294200.0,6.367900,NaN,NaN,...,NaN,0,51.152394,51.152394,NaN,NaN,7.452685,1.338568,1,1
2015-01-02,EEM,39.189999,39.209999,38.669998,38.770000,31.612551,65010000.0,-0.721861,NaN,NaN,...,NaN,0,46.267433,46.267433,NaN,NaN,6.677284,2.406311,1,1
2015-01-02,GOOGL,26.629999,26.790001,26.393999,26.477501,26.447147,26480000.0,-0.163397,NaN,NaN,...,NaN,0,42.303376,42.303376,NaN,NaN,5.013084,2.927666,1,1


In [56]:
# Threshold for significant crossover
threshold = 0.01

# Buy if MACD crosses above Signal Line with a strong difference
df_long.loc[(df_long['MACD'] - df_long['Signal_Line']) > threshold, 'MACD_Signal'] = 1

# Sell if MACD crosses below Signal Line with a strong difference
df_long.loc[(df_long['MACD'] - df_long['Signal_Line']) < -threshold, 'MACD_Signal'] = -1

In [57]:
df_long

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,...,RSI,reversal_signal,rolling_mean,time_based_rolling,upper_BB,lower_BB,MACD,Signal_Line,Momentum_Signal,MACD_Signal
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,AAPL,27.847500,27.860001,26.837500,27.332500,24.373962,212818400.0,NaN,NaN,NaN,...,NaN,0,24.373962,24.373962,NaN,NaN,0.000000,0.000000,0,0
2015-01-02,AMZN,15.629000,15.737500,15.348000,15.426000,15.426000,55664000.0,-0.367112,NaN,NaN,...,NaN,0,19.899981,19.899981,NaN,NaN,-0.949806,-0.189961,-1,-1
2015-01-02,BA,131.070007,131.839996,129.089996,129.949997,113.657219,4294200.0,6.367900,NaN,NaN,...,NaN,0,51.152394,51.152394,NaN,NaN,7.452685,1.338568,1,1
2015-01-02,EEM,39.189999,39.209999,38.669998,38.770000,31.612551,65010000.0,-0.721861,NaN,NaN,...,NaN,0,46.267433,46.267433,NaN,NaN,6.677284,2.406311,1,1
2015-01-02,GOOGL,26.629999,26.790001,26.393999,26.477501,26.447147,26480000.0,-0.163397,NaN,NaN,...,NaN,0,42.303376,42.303376,NaN,NaN,5.013084,2.927666,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498,185.516201,182.332375,...,50.394358,0,184.194446,181.769009,470.830307,-99.825305,12.561939,7.563563,1,1
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242,182.438801,179.654525,...,47.490197,0,175.586843,181.199684,463.239052,-118.218049,-3.083380,5.434175,-1,-1
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247,177.806001,180.486175,...,51.114988,0,178.316001,181.372115,468.767105,-112.135103,1.407875,4.628915,-1,-1


In [58]:
# Only record the signal when it changes
df_long['MACD_Signal'] = df_long['MACD_Signal'].diff().fillna(0)

In [60]:
df_long.tail(7)

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,...,RSI,reversal_signal,rolling_mean,time_based_rolling,upper_BB,lower_BB,MACD,Signal_Line,Momentum_Signal,MACD_Signal
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-05,SHOP,70.000000,70.930000,69.305000,70.110001,70.110001,3085582.0,0.051124,178.943201,179.941975,...,47.254265,0,168.831877,180.873824,401.138159,-67.554156,-2.369242,0.923596,-1,-2.0
2024-09-05,SPY,550.890015,553.799500,547.099976,549.609985,549.609985,44115385.0,6.839252,187.775401,182.380975,...,57.984125,0,191.230590,182.186052,473.106992,-86.618991,27.875464,6.313969,1,2.0
2024-09-05,SQ,63.970001,65.230003,63.699402,64.580002,64.580002,3340412.0,-0.882498,185.516201,182.332375,...,50.394358,0,184.194446,181.769009,470.830307,-99.825305,12.561939,7.563563,1,0.0
2024-09-05,T,20.709999,20.830000,20.520000,20.650000,20.650000,39954264.0,-0.680242,182.438801,179.654525,...,47.490197,0,175.586843,181.199684,463.239052,-118.218049,-3.083380,5.434175,-1,-2.0
2024-09-05,TSLA,223.490005,234.990005,222.250000,230.169998,230.169998,118648447.0,10.146247,177.806001,180.486175,...,51.114988,0,178.316001,181.372115,468.767105,-112.135103,1.407875,4.628915,-1,0.0
2024-09-05,V,281.000000,281.500000,277.416992,278.619995,278.619995,6447187.0,0.210497,182.074001,181.781725,...,51.044811,0,181.128001,181.713335,474.450121,-112.194120,8.775577,5.458247,1,2.0
2024-09-05,XOM,115.070000,115.279999,113.070000,113.169998,113.169998,13037121.0,-0.593820,182.912201,181.294275,...,50.714845,0,177.892001,181.473673,472.788320,-117.004319,1.249697,4.616537,-1,-2.0


In [61]:
df_man = df_long[df_long['Ticker'] == 'MANU']

In [63]:
df_man.tail()

Price,Ticker,Open,High,Low,Close,Adj Close,Volume,returns,MA50,MA200,...,RSI,reversal_signal,rolling_mean,time_based_rolling,upper_BB,lower_BB,MACD,Signal_Line,Momentum_Signal,MACD_Signal
Date,,,,,,,,,,,,,,,,,,,,,
2024-08-29,MANU,17.520000,17.730000,17.389999,17.459999,17.459999,230000.0,-0.757668,173.230800,180.617075,...,33.351188,0,139.464443,179.111870,449.212368,-104.897371,-22.412516,-9.010131,-1,0.0
2024-08-30,MANU,17.500000,17.500000,17.030001,17.230000,17.230000,239600.0,-0.762246,173.133999,180.657525,...,33.450369,0,140.927779,179.289627,450.272268,-103.662269,-22.069805,-8.458506,-1,0.0
2024-09-03,MANU,17.070000,17.430000,16.900000,17.059999,17.059999,274500.0,-0.766333,173.447600,180.553225,...,33.178853,0,137.493335,180.352899,452.840294,-107.035294,-22.904860,-9.367327,-1,0.0
2024-09-04,MANU,17.030001,17.209999,16.799999,17.139999,17.139999,757800.0,-0.763782,172.469801,180.188675,...,33.444336,0,136.971112,180.259743,445.088731,-102.650728,-22.409347,-9.181569,-1,0.0
2024-09-05,MANU,17.219999,17.325001,17.000000,17.080000,17.080000,200940.0,-0.760011,171.315601,179.835825,...,33.527513,0,137.010002,180.133519,445.709092,-102.564091,-22.416910,-8.854837,-1,0.0


## Part IV: Backtesting

In [56]:
initial_capital = 100_000  # Example initial capital
transaction_cost = 0.001  # Example transaction cost (0.1%)

In [57]:
# Create a column to hold the position (1 for holding stock, 0 for no position)
df_long['Position'] = df_long['MACD_Signal'].shift(1)  # Shift by 1 to act on next day

# Calculate daily returns
df_long['Daily_Return'] = df_long['Adj Close'].pct_change()

# Calculate strategy returns by multiplying position by daily returns
df_long['Strategy_Return'] = df_long['Position'] * df_long['Daily_Return']

# Account for transaction costs
df_long['Strategy_Return'] = df_long['Strategy_Return'] - transaction_cost * df_long['Position'].diff().abs()

# Calculate cumulative returns based on the strategy
df_long['Cumulative_Return'] = (1 + df_long['Strategy_Return']).cumprod() * initial_capital

In [58]:
total_return = df_long['Cumulative_Return'].iloc[-1] / initial_capital - 1
print(f"Total Return: {total_return * 100:.2f}%")

Total Return: -100.00%


In [59]:
risk_free_rate = 0.01  # Assume 1% annual risk-free rate

# Calculate excess returns (strategy return minus risk-free rate)
excess_return = df_long['Strategy_Return'] - risk_free_rate / 252  # Assuming 252 trading days in a year

# Calculate Sharpe Ratio
sharpe_ratio = excess_return.mean() / excess_return.std() * np.sqrt(252)
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Sharpe Ratio: -3.51


In [60]:
# Calculate rolling max of cumulative returns
rolling_max = df_long['Cumulative_Return'].cummax()

# Calculate daily drawdown
df_long['Drawdown'] = df_long['Cumulative_Return'] / rolling_max - 1

# Max drawdown
max_drawdown = df_long['Drawdown'].min()
print(f"Max Drawdown: {max_drawdown * 100:.2f}%")

Max Drawdown: -29434.27%


In [62]:
import backtrader as bt

class MACDStrategy(bt.Strategy):
    def __init__(self):
        self.macd = bt.indicators.MACD(self.data.close)
        self.signal_line = self.macd.signal
    
    def next(self):
        if self.macd.macd > self.signal_line:
            if not self.position:  # Only buy if not in position
                self.buy(size=10)
        elif self.macd.macd < self.signal_line:
            if self.position:  # Only sell if in position
                self.sell(size=10)

cerebro = bt.Cerebro()
cerebro.addstrategy(MACDStrategy)

# Add data feed
data_feed = bt.feeds.PandasData(dataname=df_long)
cerebro.adddata(data_feed)

# Set initial capital and commission
cerebro.broker.set_cash(100000)
cerebro.broker.setcommission(commission=0.001)

# Run the backtest
cerebro.run()

# Plot the results (with IPython disabled for backtrader)
cerebro.plot(iplot=False);

# Plot the results
#cerebro.plot();

<IPython.core.display.Javascript object>

In [64]:
import matplotlib.pyplot as plt

# Run the backtest
results = cerebro.run()

# Access broker's cash and value data
portfolio_value = cerebro.broker.getvalue()

# Plot the cumulative portfolio value
plt.plot(df_long.index, df_long['Cumulative_Return'], label='Strategy Returns')
plt.title('Portfolio Value Over Time')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>